In [2]:
%load_ext autoreload
%autoreload 2
from biotorch.benchmark.run import Benchmark

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from biotorch.models.weight_mirroring.resnet import resnet18, resnet34, resnet50, wide_resnet101_2

In [20]:
from collections import deque, OrderedDict, defaultdict
def compute_angles_module(module):
    queue = deque()
    layers_alignment = OrderedDict()
    seen_keys = defaultdict(lambda: 0)
    
    # First pass to store module keys
    if len(module._modules.keys()) > 0:
        for module_keys in module._modules.keys():
            queue.append((module, module_keys))
            
    # If we receive only a layer
    else:
        layer = module
        if 'alignment' in layer.__dict__:
            angle = layer.compute_alignment()
            key_name = 'layer'
            layers_alignment[key_name] = angle
            
    # Approximate depth first traversal of the model using a deque
    while len(queue) > 0:
        module, module_key = queue.popleft()
        layer = getattr(module, module_key)
        if 'alignment' in layer.__dict__:
            angle = layer.compute_alignment()
            key_name = module_key + '_' + str(seen_keys[module_key])
            seen_keys[module_key] += 1 
            layers_alignment[key_name] = angle
        if len(layer._modules.keys()) > 0:
            # Reverse list as we are appending from the left side of the queue
            for key in list(layer._modules.keys())[::-1]:
                queue.appendleft((layer, key))
                
    return layers_alignment

In [21]:
model = resnet18(pretrained=False, num_classes=10)

In [22]:
import torch
x = torch.randn((10, 3, 224, 224))

In [23]:
linear_layer

Linear(in_features=512, out_features=10, bias=True)

In [31]:
linear_layer = model.fc
noise_amplitude = 0.1
for i in range(10000):
    with torch.no_grad():
        input_noise = noise_amplitude * torch.randn(12, 512)
        output_noise = linear_layer(input_noise)
        linear_layer.update_B(input_noise, 
                            output_noise, 
                            mirror_learning_rate=0.05, 
                            growth_control=True,
                            damping_factor=0.5)
        layers_alignment = compute_angles_module(linear_layer)
        print(layers_alignment)

OrderedDict([('layer', tensor(90.0252))])
OrderedDict([('layer', tensor(90.1967))])
OrderedDict([('layer', tensor(90.6656))])
OrderedDict([('layer', tensor(90.7723))])
OrderedDict([('layer', tensor(90.6465))])
OrderedDict([('layer', tensor(90.5991))])
OrderedDict([('layer', tensor(90.6646))])
OrderedDict([('layer', tensor(90.5998))])
OrderedDict([('layer', tensor(89.3459))])
OrderedDict([('layer', tensor(88.4419))])
OrderedDict([('layer', tensor(88.9015))])
OrderedDict([('layer', tensor(89.3344))])
OrderedDict([('layer', tensor(89.8514))])
OrderedDict([('layer', tensor(89.9274))])
OrderedDict([('layer', tensor(89.6659))])
OrderedDict([('layer', tensor(89.8931))])
OrderedDict([('layer', tensor(90.4793))])
OrderedDict([('layer', tensor(90.4039))])
OrderedDict([('layer', tensor(90.7702))])
OrderedDict([('layer', tensor(90.2368))])
OrderedDict([('layer', tensor(88.4143))])
OrderedDict([('layer', tensor(88.0673))])
OrderedDict([('layer', tensor(88.5788))])
OrderedDict([('layer', tensor(88.6

OrderedDict([('layer', tensor(90.0761))])
OrderedDict([('layer', tensor(89.9459))])
OrderedDict([('layer', tensor(89.7034))])
OrderedDict([('layer', tensor(89.1671))])
OrderedDict([('layer', tensor(88.8362))])
OrderedDict([('layer', tensor(88.9848))])
OrderedDict([('layer', tensor(89.8012))])
OrderedDict([('layer', tensor(90.0098))])
OrderedDict([('layer', tensor(90.4292))])
OrderedDict([('layer', tensor(90.8434))])
OrderedDict([('layer', tensor(90.6540))])
OrderedDict([('layer', tensor(89.9804))])
OrderedDict([('layer', tensor(89.6063))])
OrderedDict([('layer', tensor(89.5407))])
OrderedDict([('layer', tensor(89.2250))])
OrderedDict([('layer', tensor(89.8629))])
OrderedDict([('layer', tensor(89.7885))])
OrderedDict([('layer', tensor(89.9344))])
OrderedDict([('layer', tensor(90.7600))])
OrderedDict([('layer', tensor(91.1535))])
OrderedDict([('layer', tensor(90.0565))])
OrderedDict([('layer', tensor(89.5653))])
OrderedDict([('layer', tensor(89.8453))])
OrderedDict([('layer', tensor(89.4

OrderedDict([('layer', tensor(88.3574))])
OrderedDict([('layer', tensor(89.1921))])
OrderedDict([('layer', tensor(88.4807))])
OrderedDict([('layer', tensor(88.9864))])
OrderedDict([('layer', tensor(89.7577))])
OrderedDict([('layer', tensor(89.6129))])
OrderedDict([('layer', tensor(89.9290))])
OrderedDict([('layer', tensor(90.1770))])
OrderedDict([('layer', tensor(90.8983))])
OrderedDict([('layer', tensor(90.5347))])
OrderedDict([('layer', tensor(90.0085))])
OrderedDict([('layer', tensor(90.5835))])
OrderedDict([('layer', tensor(90.7747))])
OrderedDict([('layer', tensor(91.4132))])
OrderedDict([('layer', tensor(91.1614))])
OrderedDict([('layer', tensor(90.0600))])
OrderedDict([('layer', tensor(90.7419))])
OrderedDict([('layer', tensor(90.6156))])
OrderedDict([('layer', tensor(89.9374))])
OrderedDict([('layer', tensor(89.3798))])
OrderedDict([('layer', tensor(89.9042))])
OrderedDict([('layer', tensor(89.8622))])
OrderedDict([('layer', tensor(89.8682))])
OrderedDict([('layer', tensor(90.1

OrderedDict([('layer', tensor(92.0919))])
OrderedDict([('layer', tensor(90.9947))])
OrderedDict([('layer', tensor(90.9194))])
OrderedDict([('layer', tensor(90.9375))])
OrderedDict([('layer', tensor(90.8883))])
OrderedDict([('layer', tensor(90.2808))])
OrderedDict([('layer', tensor(90.0282))])
OrderedDict([('layer', tensor(90.5517))])
OrderedDict([('layer', tensor(89.3473))])
OrderedDict([('layer', tensor(89.4855))])
OrderedDict([('layer', tensor(90.1172))])
OrderedDict([('layer', tensor(90.4107))])
OrderedDict([('layer', tensor(90.2611))])
OrderedDict([('layer', tensor(89.5221))])
OrderedDict([('layer', tensor(89.6112))])
OrderedDict([('layer', tensor(89.9765))])
OrderedDict([('layer', tensor(90.3678))])
OrderedDict([('layer', tensor(90.5639))])
OrderedDict([('layer', tensor(90.2553))])
OrderedDict([('layer', tensor(90.0946))])
OrderedDict([('layer', tensor(89.6773))])
OrderedDict([('layer', tensor(88.5559))])
OrderedDict([('layer', tensor(89.3148))])
OrderedDict([('layer', tensor(90.0

OrderedDict([('layer', tensor(89.4073))])
OrderedDict([('layer', tensor(89.9771))])
OrderedDict([('layer', tensor(89.5414))])
OrderedDict([('layer', tensor(89.6360))])
OrderedDict([('layer', tensor(88.7384))])
OrderedDict([('layer', tensor(89.8032))])
OrderedDict([('layer', tensor(88.9159))])
OrderedDict([('layer', tensor(88.7402))])
OrderedDict([('layer', tensor(89.9980))])
OrderedDict([('layer', tensor(90.1617))])
OrderedDict([('layer', tensor(89.9295))])
OrderedDict([('layer', tensor(89.8037))])
OrderedDict([('layer', tensor(90.4281))])
OrderedDict([('layer', tensor(89.5175))])
OrderedDict([('layer', tensor(89.1238))])
OrderedDict([('layer', tensor(89.6762))])
OrderedDict([('layer', tensor(88.9873))])
OrderedDict([('layer', tensor(89.0909))])
OrderedDict([('layer', tensor(89.8335))])
OrderedDict([('layer', tensor(90.7508))])
OrderedDict([('layer', tensor(90.3884))])
OrderedDict([('layer', tensor(91.4009))])
OrderedDict([('layer', tensor(91.6641))])
OrderedDict([('layer', tensor(90.9

...

OrderedDict([('layer', tensor(88.9580))])
OrderedDict([('layer', tensor(88.7780))])
OrderedDict([('layer', tensor(89.1179))])
OrderedDict([('layer', tensor(89.1942))])
OrderedDict([('layer', tensor(89.8220))])
OrderedDict([('layer', tensor(91.1619))])
OrderedDict([('layer', tensor(90.6332))])
OrderedDict([('layer', tensor(90.5647))])
OrderedDict([('layer', tensor(89.8670))])
OrderedDict([('layer', tensor(90.8426))])
OrderedDict([('layer', tensor(90.4363))])
OrderedDict([('layer', tensor(90.5319))])
OrderedDict([('layer', tensor(89.9081))])
OrderedDict([('layer', tensor(89.2817))])
OrderedDict([('layer', tensor(90.1665))])
OrderedDict([('layer', tensor(89.7891))])
OrderedDict([('layer', tensor(88.9113))])
OrderedDict([('layer', tensor(90.3579))])
OrderedDict([('layer', tensor(90.2302))])
OrderedDict([('layer', tensor(89.3578))])
OrderedDict([('layer', tensor(89.3526))])
OrderedDict([('layer', tensor(89.7221))])
OrderedDict([('layer', tensor(89.9331))])
OrderedDict([('layer', tensor(90.3

OrderedDict([('layer', tensor(89.3766))])
OrderedDict([('layer', tensor(89.3532))])
OrderedDict([('layer', tensor(89.4455))])
OrderedDict([('layer', tensor(90.1025))])
OrderedDict([('layer', tensor(90.2206))])
OrderedDict([('layer', tensor(90.1581))])
OrderedDict([('layer', tensor(89.9766))])
OrderedDict([('layer', tensor(89.4454))])
OrderedDict([('layer', tensor(89.3326))])
OrderedDict([('layer', tensor(89.4917))])
OrderedDict([('layer', tensor(89.1647))])
OrderedDict([('layer', tensor(89.0483))])
OrderedDict([('layer', tensor(88.6127))])
OrderedDict([('layer', tensor(88.4222))])
OrderedDict([('layer', tensor(88.8177))])
OrderedDict([('layer', tensor(89.1498))])
OrderedDict([('layer', tensor(88.7982))])
OrderedDict([('layer', tensor(88.5495))])
OrderedDict([('layer', tensor(89.2087))])
OrderedDict([('layer', tensor(89.6964))])
OrderedDict([('layer', tensor(89.4701))])
OrderedDict([('layer', tensor(89.0422))])
OrderedDict([('layer', tensor(89.7063))])
OrderedDict([('layer', tensor(88.9

OrderedDict([('layer', tensor(90.4362))])
OrderedDict([('layer', tensor(90.6645))])
OrderedDict([('layer', tensor(90.4292))])
OrderedDict([('layer', tensor(90.5241))])
OrderedDict([('layer', tensor(90.7858))])
OrderedDict([('layer', tensor(90.9051))])
OrderedDict([('layer', tensor(90.1824))])
OrderedDict([('layer', tensor(90.6310))])
OrderedDict([('layer', tensor(90.8386))])
OrderedDict([('layer', tensor(90.7227))])
OrderedDict([('layer', tensor(90.5362))])
OrderedDict([('layer', tensor(89.1210))])
OrderedDict([('layer', tensor(89.0211))])
OrderedDict([('layer', tensor(88.5651))])
OrderedDict([('layer', tensor(88.8822))])
OrderedDict([('layer', tensor(89.1850))])
OrderedDict([('layer', tensor(88.7478))])
OrderedDict([('layer', tensor(88.7025))])
OrderedDict([('layer', tensor(88.2674))])
OrderedDict([('layer', tensor(88.5232))])
OrderedDict([('layer', tensor(88.6714))])
OrderedDict([('layer', tensor(88.8621))])
OrderedDict([('layer', tensor(88.8737))])
OrderedDict([('layer', tensor(89.4

OrderedDict([('layer', tensor(89.8985))])
OrderedDict([('layer', tensor(90.3024))])
OrderedDict([('layer', tensor(90.5141))])
OrderedDict([('layer', tensor(89.8703))])
OrderedDict([('layer', tensor(89.2620))])
OrderedDict([('layer', tensor(88.6631))])
OrderedDict([('layer', tensor(88.8277))])
OrderedDict([('layer', tensor(89.4058))])
OrderedDict([('layer', tensor(88.5652))])
OrderedDict([('layer', tensor(88.3800))])
OrderedDict([('layer', tensor(89.2794))])
OrderedDict([('layer', tensor(88.5305))])
OrderedDict([('layer', tensor(89.0221))])
OrderedDict([('layer', tensor(89.2581))])
OrderedDict([('layer', tensor(89.5344))])
OrderedDict([('layer', tensor(89.9764))])
OrderedDict([('layer', tensor(90.3269))])
OrderedDict([('layer', tensor(90.1885))])
OrderedDict([('layer', tensor(89.9742))])
OrderedDict([('layer', tensor(89.9620))])
OrderedDict([('layer', tensor(89.9725))])
OrderedDict([('layer', tensor(90.3045))])
OrderedDict([('layer', tensor(90.1863))])
OrderedDict([('layer', tensor(89.9

OrderedDict([('layer', tensor(90.4601))])
OrderedDict([('layer', tensor(90.4789))])
OrderedDict([('layer', tensor(90.3346))])
OrderedDict([('layer', tensor(90.7889))])
OrderedDict([('layer', tensor(90.6704))])
OrderedDict([('layer', tensor(89.7368))])
OrderedDict([('layer', tensor(90.6116))])
OrderedDict([('layer', tensor(89.7907))])
OrderedDict([('layer', tensor(89.9067))])
OrderedDict([('layer', tensor(90.3326))])
OrderedDict([('layer', tensor(89.5087))])
OrderedDict([('layer', tensor(89.2760))])
OrderedDict([('layer', tensor(89.1010))])
OrderedDict([('layer', tensor(89.2145))])
OrderedDict([('layer', tensor(88.4000))])
OrderedDict([('layer', tensor(89.7090))])
OrderedDict([('layer', tensor(89.3157))])
OrderedDict([('layer', tensor(89.4686))])
OrderedDict([('layer', tensor(89.1869))])
OrderedDict([('layer', tensor(88.9205))])
OrderedDict([('layer', tensor(89.5811))])
OrderedDict([('layer', tensor(89.4904))])
OrderedDict([('layer', tensor(89.1996))])
OrderedDict([('layer', tensor(89.3

In [25]:
layers_alignment = compute_angles_module(model)
layers_alignment

OrderedDict([('conv1_0', tensor(90.0027)),
             ('conv1_1', tensor(90.0435)),
             ('conv2_0', tensor(90.5038)),
             ('conv1_2', tensor(89.9072)),
             ('conv2_1', tensor(90.2888)),
             ('conv1_3', tensor(90.1281)),
             ('conv2_2', tensor(90.0208)),
             ('0_0', tensor(89.1358)),
             ('conv1_4', tensor(89.8470)),
             ('conv2_3', tensor(90.2987)),
             ('conv1_5', tensor(90.0346)),
             ('conv2_4', tensor(90.0171)),
             ('0_1', tensor(90.4614)),
             ('conv1_6', tensor(90.0779)),
             ('conv2_5', tensor(89.9771)),
             ('conv1_7', tensor(90.0542)),
             ('conv2_6', tensor(89.9995)),
             ('0_2', tensor(89.9728)),
             ('conv1_8', tensor(90.0238)),
             ('conv2_7', tensor(90.0196)),
             ('fc_0', tensor(90.5137))])

In [10]:
layers_alignment = compute_angles_module(model)

In [11]:
print(layers_alignment)

OrderedDict([('conv1_0', tensor(90.6273)),
             ('conv1_1', tensor(90.6992)),
             ('conv2_0', tensor(90.4775)),
             ('conv1_2', tensor(90.2928)),
             ('conv2_1', tensor(89.5960)),
             ('conv1_3', tensor(89.8826)),
             ('conv2_2', tensor(89.9748)),
             ('0_0', tensor(90.7542)),
             ('conv1_4', tensor(89.9956)),
             ('conv2_3', tensor(89.8266)),
             ('conv1_5', tensor(90.0207)),
             ('conv2_4', tensor(90.1097)),
             ('0_1', tensor(90.2923)),
             ('conv1_6', tensor(90.0619)),
             ('conv2_5', tensor(89.8943)),
             ('conv1_7', tensor(90.0901)),
             ('conv2_6', tensor(90.0088)),
             ('0_2', tensor(89.7614)),
             ('conv1_8', tensor(89.9518)),
             ('conv2_7', tensor(90.0207)),
             ('fc_0', tensor(90.2587))])

In [100]:
from collections import defaultdict
seen_keys = defaultdict(lambda: 0)
seen_keys['conv2']

0

In [97]:
seen_keys['conv2']

NameError: name 'seen_keys' is not defined

In [95]:
layers_list

[[(Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
   biotorch.layers.weight_mirroring.conv.Conv2d)],
 [(BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   torch.nn.modules.batchnorm.BatchNorm2d)],
 [(ReLU(inplace=True), torch.nn.modules.activation.ReLU)],
 [(MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
   torch.nn.modules.pooling.MaxPool2d)],
 [(Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   biotorch.layers.weight_mirroring.conv.Conv2d)],
 [(BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   torch.nn.modules.batchnorm.BatchNorm2d)],
 [(ReLU(inplace=True), torch.nn.modules.activation.ReLU)],
 [(Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
   biotorch.layers.weight_mirroring.conv.Conv2d)],
 [(BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
   torch.nn.modules.ba

In [ ]:
queue.append()

In [29]:
x = [] 
compute_angle_layers(model, x)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
tensor(90.7209)
Linear(in_features=512, out_features=10, bias=True)
tensor(89.3565)
conv1
bn1
relu
maxpool
layer1
0
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.0204)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.1279)
conv1
bn1
relu
conv2
bn2
relu2
1
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.3268)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.2096)
conv1
bn1
relu
conv2
bn2
relu2
layer2
0
Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
tensor(90.0323)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.0547)
conv1
bn1
relu
conv2
bn2
relu2
downsample
Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.5946)
0
1
1
Conv2d(128, 128, kernel_size=(3, 3), str

In [27]:
sum(x)

21

In [7]:
compute_angle_layers(model)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
tensor(90.7209)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.0204)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.1279)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.3268)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.2096)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.0204)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.1279)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.3268)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.2096)
Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
tensor(90.0323)
Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=Fals

tensor(89.8488)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9422)
Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
tensor(89.8325)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(90.0091)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(89.7005)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9091)
Conv2d(256, 256, kernel_size=(3, 3), str

tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9910)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9887)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9743)
Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
tensor(89.9938)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), str

tensor(89.9887)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9743)
Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
tensor(89.9938)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9910)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
tensor(90.2159)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
tensor(89.9887)
Conv2d(512, 512, kernel_size=(3, 3), str

In [6]:
import torch
import torch.nn.functional as F
x = torch.randn(1, 3, 50, 50)
# define the W and B kernels
W = torch.randn([64, 3, 7, 7])
B = torch.randn([64, 3, 7, 7])
# run in the forward pass
stride = [3, 3]
padding = 3
y = F.conv2d(x, W, stride=stride, padding=padding)
# compute dB
dW = F.conv2d(torch.transpose(x, 0, 1), torch.transpose(y, 0, 1), dilation=stride, padding=padding)
dB = torch.transpose(F.interpolate(dW, size=[W.shape[2], W.shape[3]]), 0, 1)
print("x  = ", x.shape)
print("y  = ", y.shape)
print("B  = ", B.shape)
print("dW = ", dW.shape)
print("dB = ", dB.shape)

x  =  torch.Size([1, 3, 50, 50])
y  =  torch.Size([1, 64, 17, 17])
B  =  torch.Size([64, 3, 7, 7])
dW =  torch.Size([3, 64, 8, 8])
dB =  torch.Size([64, 3, 7, 7])


In [11]:
import torch
import torch.nn.functional as F
# define the W and B kernels
forward_kernel = torch.randn([16, 16, 3, 3])
backward_kernel = torch.randn([16, 16, 3, 3])
# downsample in the forward pass
noise_input  = torch.randn(1, 16, 12, 12)
noise_output = F.conv2d(noise_input, forward_kernel, stride=[1,1], padding=1)
# upsample in the backward pass
output = F.conv_transpose2d(noise_output, backward_kernel, stride=[1,1], padding=1)
assert(list(output.shape) == list(noise_input.shape))

In [12]:
output.size()

torch.Size([1, 16, 12, 12])

In [13]:
noise_input.size()

torch.Size([1, 16, 12, 12])

In [4]:
benchmark = Benchmark('/home/albert/projects/biotorch/biotorch/configs/mnist.yaml')

In [5]:
benchmark.run()

Preparing CIFAR10 Dataset and storing data in ./data
Files already downloaded and verified
Files already downloaded and verified
=> Creating model 'resnet18'
Converting ResNet-18 to Feedback Alignment mode


TypeError: super(type, obj): obj must be an instance or subtype of type

In [23]:
import torch
import torch.nn.functional as F
x = torch.randn(1, 3, 224, 224)
# define the W and B kernels
W = torch.randn([64, 3, 7, 7])
B = torch.randn([64, 3, 7, 7])
# downsample in the forward pass
y = F.conv2d(x, W, stride=2, padding=3)
# compute the dB update for B
dB = F.conv2d(torch.transpose(x, 0, 1), torch.transpose(y, 0, 1), stride=2, padding=3)

dB = torch.transpose(dB, 0, 1)
print("x t = ", torch.transpose(x, 0, 1).shape)
print("y t = ", torch.transpose(y, 0, 1).shape)
print("x  = ", x.shape)
print("y  = ", y.shape)
print("B  = ", B.shape)
print("dB = ", dB.shape)

x t =  torch.Size([3, 1, 224, 224])
y t =  torch.Size([64, 1, 112, 112])
x  =  torch.Size([1, 3, 224, 224])
y  =  torch.Size([1, 64, 112, 112])
B  =  torch.Size([64, 3, 7, 7])
dB =  torch.Size([64, 3, 60, 60])


In [4]:
m = benchmark.model

In [5]:
def compute_angle_layers(module):
        # Go through all of module nn.module (e.g. network or layer)
        for module_name in module._modules.keys():
            # Get layer
            layer = getattr(module, module_name)
            # Convert layer
            print(layer)
        # Iterate through immediate child modules
        for name, child_module in module.named_children():
            forward_layers(child_module)

In [6]:
forward_layers(m)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
biotorch.layers.fa

<module 'biotorch.layers.fa' from '/home/albert/projects/biotorch/biotorch/layers/fa/__init__.py'>

In [4]:
benchmark.model

BioModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [6]:
from torchvision import models

In [7]:
model = models.resnet18()

In [8]:
import torch

In [10]:
model = torch.nn.DataParallel(model, [1, 2])

In [14]:
torch.save(model.module, 'model.pth')

In [15]:
torch.load('model.pth')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  